## 1. multiprocessing.Process() Objects and Methods:
#### 1.1 start()
- Starts the process by invoking the run() method in a separate memory space.

#### 1.2 run()
- Contains the code to be executed by the process. By default, it calls the target function.

#### 1.3 join(timeout=None)
- Blocks the calling thread until the process completes or the optional timeout expires.

#### 1.4 is_alive()
- Returns True if the process is currently running.

#### 1.5 terminate()
- Immediately terminates the process without cleanup.

#### 1.6 kill()
- Forcefully kills the process (available from Python 3.7).

#### 1.7 exitcode
- Returns the exit status of the process (e.g., 0 for success, None if not yet finished).

#### 1.8 daemon
- Boolean attribute that specifies whether the process runs as a daemon.

#### 1.9 pid
- The Process ID (PID) assigned by the operating system once the process starts.

#### 1.10 name
- The name of the process, useful for debugging or logging.

In [1]:
from multiprocessing import Process
import os
import time

# Target function for the process
def worker_task(name):
    print(f"Process {name} started with PID: {os.getpid()}")
    time.sleep(2)
    print(f"Process {name} finished.")

# Create a process
process = Process(target=worker_task, args=("TestProcess",), name="MyProcess")

# 1. Start the process
print(f"Starting process: {process.name}")
process.start()

# 2. Check if the process is alive
print(f"Is process alive? {process.is_alive()}")

# 3. Get process attributes
print(f"Process Name: {process.name}")
print(f"Process ID (PID): {process.pid}")  # PID will be None until process starts

# 4. Wait for process to complete
process.join()

# 5. Check exit status
print(f"Process Exit Code: {process.exitcode}")
print(f"Is process alive after completion? {process.is_alive()}")


Starting process: MyProcess
Process TestProcess started with PID: 5659
Is process alive? True
Process Name: MyProcess
Process ID (PID): 5659
Process TestProcess finished.
Process Exit Code: 0
Is process alive after completion? False


### Why Do We Call start() Instead of run() in Multiprocessing?
#### i. Using start() Method

In [2]:
from multiprocessing import Process
import os
import time

def task():
    for i in range(5):
        print(f"Task running in process: {os.getpid()}, iteration: {i}")
        time.sleep(1)

if __name__ == "__main__":
    # Create a process and start it
    process = Process(target=task)
    process.start()
    process.join()

    # Main process continues its own execution
    for i in range(3):
        print(f"Main process: {os.getpid()}, iteration: {i}")
        time.sleep(1)


Task running in process: 5769, iteration: 0
Task running in process: 5769, iteration: 1
Task running in process: 5769, iteration: 2
Task running in process: 5769, iteration: 3
Task running in process: 5769, iteration: 4
Main process: 5405, iteration: 0
Main process: 5405, iteration: 1
Main process: 5405, iteration: 2


### What Happens Internally?
a. Process Creation
- process = Process(target=task) creates a new Process object and stores:
- The task function in its _target attribute.
- A unique process identifier (PID) is assigned upon creation (but not started yet).

b. Calling start()

process.start():
- Spawns a new child process.
- Allocates resources like memory and CPU for the child process.
- The operating system assigns a new PID for the child process.
- The process transitions to the RUNNABLE state and is ready to run.

c. Scheduler Picks the Process
- The operating system's process scheduler picks the child process for execution.
- The parent and child processes now execute concurrently.

d. Calling run() Automatically
- Internally, the start() method calls the run() method of the child process.
- The run() method executes the task() function in the child process's memory space.

e. Parallel Execution
- The task() function runs in the child process (separate memory space).
- Meanwhile, the main process continues executing its code independently.


#### ii. Using run() Method

In [3]:
from multiprocessing import Process
import os
import time

def task():
    for i in range(5):
        print(f"Task running in process: {os.getpid()}, iteration: {i}")
        time.sleep(1)

if __name__ == "__main__":
    # Create a process and call run() directly
    process = Process(target=task)
    process.run()  # Directly calling run()

    # Main process execution
    for i in range(3):
        print(f"Main process: {os.getpid()}, iteration: {i}")
        time.sleep(1)


Task running in process: 5405, iteration: 0
Task running in process: 5405, iteration: 1
Task running in process: 5405, iteration: 2
Task running in process: 5405, iteration: 3
Task running in process: 5405, iteration: 4
Main process: 5405, iteration: 0
Main process: 5405, iteration: 1
Main process: 5405, iteration: 2


### What Happens Internally?
a. Process Creation
- Similar to the start() example, process = Process(target=task) creates a Process object and stores the task function in _target.

b. Calling run()
- process.run() does not spawn a new process.
- Instead, the run() method executes the task() function in the main process's memory space.
- No new process is created, and no separate PID is assigned.

c. Blocking Execution
- Since the task() function runs in the main process, it blocks the main process.
- The main process cannot continue executing its own code (e.g., its loop) until task() finishes.

d. No Process Isolation
- The run() method shares the same memory space as the main process.
- This defeats the purpose of multiprocessing, as there’s no parallel execution.